In [1]:
FILENAME = unicode('{} prior.csv'.format('金时UP+'), 'utf-8')
STAR_RATE = {
    'SR': 3,
    'SSR': 1
}
UP_RATE = {
    'SR': .65,
    'SSR': .65
}
INIT_TEN_CARD = ['骑士迪昂', '玉藻猫', '玛尔达', '赫拉克勒斯', '伊丽莎白·巴托里',
                 '斯忒诺', '卡米拉', '玛丽·安托瓦内特', '卫宫', '齐格飞']
BASIC_POOL_CARD = ['南丁格尔', '弗朗西斯·德雷克', '俄里翁', '莫德雷德', '迦尔纳',
                   '玉藻前', '贞德', '弗拉德三世', '阿尔托莉雅·潘德拉贡', '阿蒂拉',
                   '阿周那', '诸葛孔明〔埃尔梅罗Ⅱ世〕', '开膛手杰克',
                   '阿塔兰忒', '安妮·伯妮＆玛莉·瑞德', '阿斯托尔福', '弗兰肯斯坦', '童谣',
                   '贝奥武夫', '芬恩·麦克库尔', '兰斯洛特', '卫宫〔Assassin〕', '罗摩',
                   '海伦娜·布拉瓦茨基',
                   ] + INIT_TEN_CARD
UP_POOL_CARD = ['坂田金时']
POOL_NAME, POOL_CARD = map(list, zip(*[['初始十连', INIT_TEN_CARD],
                                       ['初始非保底', INIT_TEN_CARD +
                                        ['阿塔兰忒', '安妮·伯妮＆玛莉·瑞德', '阿斯托尔福', '弗兰肯斯坦', '童谣',
                                         '贝奥武夫', '芬恩·麦克库尔', '兰斯洛特', '卫宫〔Assassin〕', '罗摩',
                                         '海伦娜·布拉瓦茨基']],
                                       ['UP池', BASIC_POOL_CARD + UP_POOL_CARD],
                                       ['-1章剧情池', BASIC_POOL_CARD]]))
for pn, pc in [['0~1章剧情池', ['阿尔托莉雅·潘德拉贡〔Alter〕']], ['2章剧情池', ['尼禄·克劳狄乌斯']],
               ['3章剧情池', ['美狄亚〔Lily〕']], ['4章剧情池', ['尼古拉·特斯拉', '阿尔托莉雅·潘德拉贡〔Alter〕']],
               ['5~章剧情池', ['库·丘林[Alter]', '女王梅芙', '托马斯·爱迪生', '李书文']]]:
    POOL_NAME.append(pn)
    POOL_CARD.append(POOL_CARD[-1] + pc)
SSR_CARD = {'南丁格尔', '弗朗西斯·德雷克', '俄里翁', '莫德雷德', '迦尔纳',
            '玉藻前', '贞德', '弗拉德三世', '阿尔托莉雅·潘德拉贡', '阿蒂拉',
            '阿周那', '诸葛孔明〔埃尔梅罗Ⅱ世〕', '开膛手杰克',
            '尼古拉·特斯拉', '库·丘林[Alter]', '女王梅芙', '坂田金时'}

In [2]:
from __future__ import division
import pandas as pd
import numpy as np
import collections as co
import operator as op

pn2i = dict(zip(POOL_NAME, range(len(POOL_NAME))))
i2c = list(set(reduce(op.add, POOL_CARD)))
c2i = dict(zip(i2c, range(len(i2c))))
c2s = dict(zip(i2c, map(lambda c: 'SSR' if c in SSR_CARD else 'SR', i2c)))
s2n_ = co.Counter(map(c2s.get, UP_POOL_CARD))

In [3]:
prior = np.zeros(map(len, [pn2i, c2i]))
for i in range(len(POOL_NAME)):
    s2n = co.Counter(map(c2s.get, POOL_CARD[i]))
    for c in POOL_CARD[i]:
        if POOL_NAME[i] != 'UP池' or s2n_[c2s[c]] == 0:
            prior[i, c2i[c]] += STAR_RATE[c2s[c]] / s2n[c2s[c]]
        elif c in UP_POOL_CARD:
            prior[i, c2i[c]] += STAR_RATE[c2s[c]] / s2n_[c2s[c]] * UP_RATE[c2s[c]]
        else:
            prior[i, c2i[c]] += STAR_RATE[c2s[c]] / (s2n[c2s[c]] - s2n_[c2s[c]]) * (1 - UP_RATE[c2s[c]])

In [4]:
reduce(pd.merge, map(lambda pn: pd.DataFrame(
    {'info': i2c, pn: prior[pn2i[pn]] / prior[pn2i[pn]].sum()}), POOL_NAME)).to_csv(FILENAME, index=False)#, float_format='%.15f')
pd.read_csv(FILENAME)

,info,初始十连,初始非保底,UP池,-1章剧情池,0~1章剧情池,2章剧情池,3章剧情池,4章剧情池,5~章剧情池
0,赫拉克勒斯,0.1,0.047619,0.035714,0.035714,0.034091,0.032609,0.031250,0.030000,0.027778
1,库·丘林[Alter],0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.015625
2,童谣,0.0,0.047619,0.035714,0.035714,0.034091,0.032609,0.031250,0.030000,0.027778
3,齐格飞,0.1,0.047619,0.035714,0.035714,0.034091,0.032609,0.031250,0.030000,0.027778
4,伊丽莎白·巴托里,0.1,0.047619,0.035714,0.035714,0.034091,0.032609,0.031250,0.030000,0.027778
5,李书文,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.027778
6,阿塔兰忒,0.0,0.047619,0.035714,0.035714,0.034091,0.032609,0.031250,0.030000,0.027778
7,阿蒂拉,0.0,0.000000,0.006731,0.019231,0.019231,0.019231,0.019231,0.017857,0.015625
8,阿斯托尔福,0.0,0.047619,0.035714,0.035714,0.034091,0.032609,0.031250,0.030000,0.027778
9,弗兰肯斯坦,0.0,0.047619,0.035714,0.035714,0.034091,0.032609,0.031250,0.030000,0.027778
